<a href="https://colab.research.google.com/github/pedroAndrad1/validacao_de_modelos/blob/master/Machine_Learning_Validac%CC%A7a%CC%83o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Métodos para validar um modelo

Por enquanto, apenas sabemos a acurácia como método de avaliar um modelo. Vamos ver outras formas.

In [2]:
import pandas as pd

uri = "https://gist.githubusercontent.com/guilhermesilveira/e99a526b2e7ccc6c3b70f53db43a87d2/raw/1605fc74aa778066bf2e6695e24d53cf65f2f447/machine-learning-carros-simulacao.csv"
dados = pd.read_csv(uri).drop(columns=["Unnamed: 0"], axis=1)
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano
0,30941.02,1,18,35085.22134
1,40557.96,1,20,12622.05362
2,89627.50,0,12,11440.79806
3,95276.14,0,3,43167.32682
4,117384.68,1,4,12770.11290


In [3]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

x = dados[["preco", "idade_do_modelo","km_por_ano"]]
y = dados["vendido"]

SEED = 158020
np.random.seed(SEED)
treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25,
                                                         stratify = y)
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))


Treinaremos com 7500 elementos e testaremos com 2500 elementos


In [4]:
from sklearn.dummy import DummyClassifier

dummy_stratified = DummyClassifier()
dummy_stratified.fit(treino_x, treino_y)
acuracia = dummy_stratified.score(teste_x, teste_y) * 100

print("A acurácia do dummy stratified foi %.2f%%" % acuracia)

A acurácia do dummy stratified foi 50.96%


In [5]:
from sklearn.tree import DecisionTreeClassifier

SEED = 158020
np.random.seed(SEED)
modelo = DecisionTreeClassifier(max_depth=2)
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print("A acurácia foi %.2f%%" % acuracia)


A acurácia foi 71.92%


Acúracia do nosso modelo foi essa. Mas e se passarmos um SEED diferente, tanto na hora de splitar quanto na hora de criar o modelo.

In [6]:
SEED = 5
np.random.seed(SEED)
treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25,
                                                         stratify = y)
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))

SEED = 5
np.random.seed(SEED)
modelo = DecisionTreeClassifier(max_depth=2)
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print("A acurácia foi %.2f%%" % acuracia)

Treinaremos com 7500 elementos e testaremos com 2500 elementos
A acurácia foi 76.84%


Tivemos um resultado diferente. Essa diferença no resultado pode ser o que decide entre usar o modelo ou não, essa decisão não pode ficar a mercê da aleatoriade.

#Cross Validation 

        Para ficarmos menos dependentes de um seed aleatório, podemos usar o métode de cross validation. Que consiste em separar as features em K pedaços e fazemos treino e teste K vezes cada vez escolhendo um dos grupos para teste e o resto ficando para treino. Existe uma função do sklearn.model_selection chamada cross_validate.

docs:
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html

        Essa função vai nos retornar um dicionário em que um dos itens é uma lista com as K acurácias.

        Queremos um interavlo, então iremos usar um técnica de estástica:

Intervalo = [ média - 2 * desvio padrão, média + 2 * desvio padrão ]

        A gente multiplica as pontas do intervalo por 100 para ficar no formato de porcentagem.

        Os array's do numpy já tem funções para calcular média, mean( ), e desvio padrão, std(). 

In [7]:
from sklearn.model_selection import cross_validate

#Vamos instanciar o modelo de novo.
SEED = 5
np.random.seed(SEED)
modelo = DecisionTreeClassifier(max_depth=2)

#Aplicando o cross validation com 5 grupos.

results = cross_validate(modelo, x, y, cv = 5)

results


{'fit_time': array([0.00967193, 0.01080489, 0.00854254, 0.00877976, 0.00992656]),
 'score_time': array([0.00202441, 0.00190067, 0.00168276, 0.00188661, 0.00187755]),
 'test_score': array([0.756 , 0.7565, 0.7625, 0.7545, 0.7595])}

In [8]:
#Tirando a media dos testes
media = results['test_score'].mean()
#Tirando o desvio padrao
desvio_padrao = results['test_score'].std()

print("Accuracy com cross validation, 5 = [%.2f, %.2f] " % ( (media - 2*desvio_padrao) * 100, (media + 2*desvio_padrao) * 100 ) )


Accuracy com cross validation, 5 = [75.21, 76.35] 


Perceba que ainda ficamos dependendo da quantidade de grupos que usamos para o cross validation.

In [9]:
results = cross_validate(modelo, x, y, cv = 7)

#Tirando a media dos testes
media = results['test_score'].mean()
#Tirando o desvio padrao
desvio_padrao = results['test_score'].std()

print("Accuracy com cross validation, 7 = [%.2f, %.2f] " % ( (media - 2*desvio_padrao) * 100, (media + 2*desvio_padrao) * 100 ) )


Accuracy com cross validation, 7 = [74.58, 76.98] 


Há artigos que a própria documentação cita e outros por aí na literatura dizendo que o número de grupos ideal é entre 5 e 10. Da uma lida lá.

#Splitter Classes

        Eu gostaria de embaralhar os dados antes de quebrá-los para o cross validation. Aconte que o cross_validate não tem uma opção de shuffle, mas se você ler na documentação verá que o argumento cv aceita um inteiro ou um cross-validation generator.
        
        Um cross-validation generator é uma função que é capaz de fazer o shuffle dos dados. Existe um módulo no model_selection que é chamado Splitter Classes. Neste módulo há vários cross-validation generator's. Usaremos o KFold.
        
docs:
        
   https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html

In [0]:
#Vou colocar o processe de imprimrir o intevalo em uma funçao

def imprime_results(results):
  #Tirando a media dos testes
  media = results['test_score'].mean()
  #Tirando o desvio padrao
  desvio_padrao = results['test_score'].std()
  
  print("Accuracy média: %.2f" % (media * 100) )
  print("Accuracy com cross validation: [%.2f, %.2f] " % ( (media - 2*desvio_padrao) * 100, (media + 2*desvio_padrao) * 100 ) )

In [15]:
from sklearn.model_selection import KFold
SEED = 100
np.random.seed(SEED)

#O KFold tem random_state
cv =  KFold(n_splits = 5, shuffle = True)

#Vamos instanciar o modelo de novo.

modelo = DecisionTreeClassifier(max_depth=2)

#Aplicando o cross validation com o KFold.

results = cross_validate(modelo, x, y, cv = cv)

imprime_results(results)

Accuracy média: 75.78
Accuracy com cross validation: [74.50, 77.06] 


#Stratified KFold

        Note na documentação do KFold que ele não stratificar as features por classe. Ou seja, embora haja um shuffle das features antes de separar em grupos, ainda é possível que haja um desbalanceamento entre as features de diferentes classes. Isso seria péssimo para o modelo, pois ele treinaria com as features predominantemente de uma classe e testaria com de outra, dando uma acurácia terrível.
         
         O StratifiedKFold por padrão já faz esse balanceamento. Vamos usá-lo.
         
docs:
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html#sklearn.model_selection.StratifiedKFold

In [12]:
from sklearn.model_selection import StratifiedKFold
SEED = 100
np.random.seed(SEED)

#O StratifiedKFold tem random_state.
#Nao vou fazer um shuffle das features antes de separa-las, vamos ver
#como se sai a cross validation.
cv =  StratifiedKFold(n_splits = 5, shuffle = False)

#Vamos instanciar o modelo de novo.

modelo = DecisionTreeClassifier(max_depth=2)

#Aplicando o cross validation com o StratifiedKFold.

results = cross_validate(modelo, x, y, cv = cv)

imprime_results(results)

Accuracy média: 75.78
Accuracy com cross validation: [75.21, 76.35] 


Veja que mesmo sem o shuffle, tivemos um bom intervalo. Nesse caso, as features ajudaram também, pois elas não são tão desbalanceadas. Mas se fossem, o StratifiedKFold, dando um shuffle até que ajuda, poderia ser uma boa solução. Tudo essas situações e quais classes usar são descritas na documentação do Sklearn.

docs:
https://scikit-learn.org/stable/modules/cross_validation.html

#Agrupamento

        Até agora, fizemos a cross validation pensando apenas na classe vendido. Ou seja, em todas as situações de treino e teste, essas classes estavam presentes em ambas. 
        
        Mas, pense o seguinte, estamos trabalhando com carros e cada carro tem um modelo que não está descrito nos nossos dados. Se tivesse em cada treino e teste no cross validation, estáriamos, provavelmente, trabalhando como todos os modelos em ambas as situações de treino e teste. 
        
        Não teria uma situação na qual o modelo treina com um conjunto de modelos e treina com features de um modelo que não estava presente nos treinos. É imprescendível que o modelo seja avaliado nesse tipo de situação. Pois no mundo real isso é muito comum, temos features de derteminados grupos, nesse caso modelos de carros, e chega novas features de grupos não prevíos no conjunto de dados.
        
        Logo, vamos criar uma nova coluna chamada modelos. Essa colunas serão números aleatórios. Mas só isso não é o suficiente, eu quero que exista a possibilidade de até 5 modelos por um determinado ano. Então, gerarei um número aleatório entre -2 e 2 e somarei este número a idade do modelo. Para isso usarei a função randint() do numpy.
        
         Para fazer a cross validation por grupos, vamos usar o classe GroupKFold.
 
 docs:
https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.randint.html
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GroupKFold.html

In [22]:
#Criando a coluna de modelos.
#A funcao randint tem um parametro que diz quantos numeros aleatorios eu quero.
#Vou colocar esse numero igual a quantidde de features que temos. Usando o len().

dados['modelo'] = dados.idade_do_modelo - np.random.randint(-2, 3, len(dados))
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano,modelo
0,30941.02,1,18,35085.22134,16
1,40557.96,1,20,12622.05362,22
2,89627.50,0,12,11440.79806,14
3,95276.14,0,3,43167.32682,4
4,117384.68,1,4,12770.11290,5


Feito isso, vamos ver quais são os números de modelos que temos usando a função unique()

In [23]:
dados.modelo.unique()

array([16, 22, 14,  4,  5, 12, 17, 13,  0,  7,  9,  6, 19, 20,  3, 18, 15,
       21, 11, 10,  8,  2,  1, -1])

Opa, temos -1 e 0. Isso é estranho para modelo de carro. Então vou somar + 2 em toda a coluna.

In [26]:
dados.modelo = dados.modelo + 2
dados.modelo.unique()

array([22, 28, 20, 10, 11, 18, 23, 19,  6, 13, 15, 12, 25, 26,  9, 24, 21,
       27, 17, 16, 14,  8,  7,  5])

Vamos ver também quantas vezes cada modelo aparece, usando a função value_counts().

In [27]:
dados.modelo.value_counts()

24    894
23    827
22    766
21    720
25    700
20    676
19    630
26    578
18    554
17    547
16    514
15    418
14    375
27    353
13    297
12    276
11    231
28    211
10    175
9     111
8      74
7      46
6      20
5       7
Name: modelo, dtype: int64

Já temos a coluna de modelos. Agora vamos vamos fazer o cross validation baseado neles.
Para dizer em quais grupos o GroupKFold deve se basear, temos que passar a coluna modelos no parâmetro groups na função de cross_validate.


In [29]:
from sklearn.model_selection import GroupKFold
SEED = 100
np.random.seed(SEED)


cv =  GroupKFold(n_splits = 5)

#Vamos instanciar o modelo de novo.

modelo = DecisionTreeClassifier(max_depth=2)

#Aplicando o cross validation com o GroupKFold.

results = cross_validate(modelo, x, y, cv = cv, groups = dados.modelo)

imprime_results(results)

Accuracy média: 75.78
Accuracy com cross validation: [73.35, 78.22] 


Nesse caso, obtemos uma avaliação parecida com a anterior porque fazer os predict's levando em consideração os modelos ou não acaba que não faz tanta diferança. Mas existem situações em que faz.

#Pipeline

        Vamos usar outro estimador, nesse caso o SVC, para fazer o cross validation. Aqui teremos um problema. Pois será necessário reescalar as features durante o cross validation. Não posso simplesmente reescalar todas as features antes do cross validation, porque, no pré processamento de dados, quando escalamos as features, fit() é aplicado somente nos dados de treino, e aqui o fit() acontece em treino e teste. Até porque faremos K separações baseadas nos grupos, para treinar e testar. Ou seja, não faremos o fit() somente uma vez. Devemos fazer esse fit() diversas vezes. Na verdade, deveríamos intercalar, fazendo uma vez esse fit, uma vez a validação, uma vez o fit, outra vez a validação, com outro conjunto de dados. 
        
        Para isso, usaremos a classe Pipeline. Esta classe é um processo e um processo tem passos. Então o primeiro passo desse processo será o StandarScaler, para rescalonar as features, e o segundo passo será a criação do modelo, SVC. Note, pela documentação, que o Pipeline funciona como um estimador. Logo, posso passa-lo como estimador para o cross_validate e a cada validação, será aplicado esse processo.
        
docs:

https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html


Obs: Essa situação está descrita na documentação do sklearn. Já coloquei o link anteriormente, mas colocarei aqui de novo.

https://scikit-learn.org/stable/modules/cross_validation.html

In [32]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline

scaler = StandardScaler()
modelo = SVC()

pipeline = Pipeline([ ('Reescalonamento', scaler), ('Instanciação do modelo', modelo )])
pipeline

Pipeline(memory=None,
         steps=[('Reescalonamento',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('Instanciação do modelo',
                 SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
                     decision_function_shape='ovr', degree=3,
                     gamma='auto_deprecated', kernel='rbf', max_iter=-1,
                     probability=False, random_state=None, shrinking=True,
                     tol=0.001, verbose=False))],
         verbose=False)

In [33]:
cv =  GroupKFold(n_splits = 5)

#Aplicando o cross validation com o GroupKFold.

results = cross_validate(pipeline, x, y, cv = cv, groups = dados.modelo)

imprime_results(results)

Accuracy média: 76.54
Accuracy com cross validation: [74.62, 78.47] 
